In [170]:
import pandas as pd
import numpy as np

In [171]:
# RENT PRICES DATABASE

In [210]:
df_rent=pd.read_csv("../datasets/2017_lloguer_preu_trim.csv")

df_rent_colnames= ["Year","Trimester","District Code","District","Neighborhood Code",
                   "Neighborhood","Average Rent", "Price"]
df_rent.columns=df_rent_colnames

df_rent["Average Rent"]=df_rent["Average Rent"].replace("Lloguer mitjà mensual (Euros/mes)","Euros per month")
df_rent["Average Rent"]=df_rent["Average Rent"].replace("Lloguer mitjà per superfície (Euros/m2 mes)","Euros/m2 per month")

df_rent=df_rent.groupby(["District","Average Rent"],as_index=False).agg({"Price":"mean"})
df_rent = df_rent.groupby(["District",'Average Rent'])['Price'].mean().unstack()

df_rent=df_rent.drop(["Euros per month"], axis=1)

df_rent["Euros/m2 per month"] = df_rent["Euros/m2 per month"].round(2)
df_rent = df_rent.sort_values(by=["Euros/m2 per month"], ascending=False)

df_rent

Average Rent,Euros/m2 per month
District,
Sarrià-Sant Gervasi,15.95
Ciutat Vella,15.91
Les Corts,14.59
Gràcia,13.51
Eixample,13.34
Sant Martí,13.03
Sants-Montjuïc,12.57
Horta-Guinardó,11.83
Sant Andreu,10.76


In [173]:
# IMMIGRANTS/GENDER DATABASE

In [190]:
#importing csv
df_gender_colnames = ["Year", "District code", "District name", "Neighbourhood code", "Neighbourhood name", "Gender", "Number immigrants"]
df_gender = pd.read_csv("../datasets/2017_immigrants_sexe.csv", names=df_gender_colnames, header=0)

#drop unnecessary columns
cols_to_drop = ["Year", "District code", "Neighbourhood code"]
df_gender2 = df_gender.drop(cols_to_drop, axis=1)

#deleting duplicate records
df_gender2 = df_gender2.drop_duplicates()

#rename gender/ensure names are uniform
df_gender2["Gender"].value_counts()
df_gender2["Gender"] = np.where((df_gender2["Gender"]=="Homes"), "Male", "Female")

# Pivoting tables to get population numbers as columns
df_gender3 = pd.pivot_table(df_gender2, index="Neighbourhood name", columns="Gender", values="Number immigrants")

# Aggregating male and female values
df_gender_last = pd.merge(df_gender2[["District name", "Neighbourhood name"]], df_gender3, on="Neighbourhood name")
df_gender_last = df_gender_last.rename(columns={"Female":"Female immigrants", "Male":"Male immigrants"})
df_gender_last = df_gender_last.drop_duplicates()


In [175]:
# POPULATION DATABASE

In [176]:
#importing csv
df_pop_colnames = ["Year", "District code", "District name", "Neighbourhood code", "Neighbourhood name", "Gender", "Age", "Number population"]
df_pop = pd.read_csv("../datasets/population.csv", names=df_pop_colnames, header=0)

#deleting duplicate records
df_pop = df_pop.drop_duplicates()

#drop columns
cols_to_drop = ["District code", "Neighbourhood code"]
df_pop2 = df_pop.drop(cols_to_drop, axis=1)

#select records from 2017 only
df_pop2 = df_pop2[df_pop2["Year"]==2017]

#aggregate numbers by neighbourhood and gender, discarding age
df_pop3 = df_pop2.groupby(["District name", "Neighbourhood name", "Gender"], as_index=False).agg({"Number population":"sum"})

df_pop4 = pd.pivot_table(df_pop3, index=["District name","Neighbourhood name"], columns="Gender", values="Number population")

df_pop_last = pd.merge(df_pop2[["District name", "Neighbourhood name"]], df_pop4, on="Neighbourhood name")
df_pop_last = df_pop_last.rename(columns={"Female":"Female total", "Male":"Male total"})
df_pop_last = df_pop_last.drop_duplicates()


In [177]:
# MERGING IMMIGRANTS/GENDER AND POPULATION TOGETHER

In [189]:
# Note: record for "No consta" from "df_gender_last" is dropped on the merge.
# This does not affect the overall reasults as only 2 individuals were listed under "No consta".
df_total = pd.merge(df_gender_last, df_pop_last, on=["District name", "Neighbourhood name"])
df_total = df_total.drop_duplicates()

# sum by district
df_aux = df_total.groupby('District name')['Female immigrants','Male immigrants','Female total','Male total'].sum()

# sum by gender
df_aux["#Immigrants"] = df_aux["Female immigrants"] + df_aux["Male immigrants"]
df_aux["#Inhabitants"] = df_aux["Female total"] + df_aux["Male total"]

# drop old columns
df_population = df_aux.drop(['Female immigrants','Male immigrants','Female total','Male total'], axis=1)

# Get percentage of immigrants per district
df_population["Percentage of immigrants"] = (df_population["#Immigrants"]*100/df_population["#Inhabitants"]).round(2)
df_population[["#Immigrants", "#Inhabitants", "Percentage of immigrants"]]
df_population.sort_values(by=["Percentage of immigrants"], ascending=False)

df_population

,#Immigrants,#Inhabitants,Percentage of immigrants
District name,,,
Ciutat Vella,12611,98232,12.84
Eixample,19047,258152,7.38
Gràcia,7254,117227,6.19
Horta-Guinardó,7799,161921,4.82
Les Corts,4375,79888,5.48
Nou Barris,8274,159214,5.20
Sant Andreu,6335,143246,4.42
Sant Martí,12720,227989,5.58
Sants-Montjuïc,11683,174478,6.70


In [179]:
# IMMIGRANTS BY NATIONALITY DATABASE

In [197]:
immigrants = pd.read_csv("../datasets/immigrants_by_nationality.csv")

# There are no missing values in the immigrants dataset.
# null_immigrants = immigrants.isnull().sum()
# print(null_immigrants)

# Types are correct:
# immigrants.dtypes

# Checking all the Nationalities of the file
# nationality = immigrants["Nationality"].value_counts()
# pd.set_option("display.max_rows", 200)
# print(nationality)

# Checking the districts in the file
# immigrants["District Name"].value_counts()

# Drop "Year" and "District code" column
immigrants_2017 = immigrants.drop(["Year"], axis=1)
immigrants_2017 = immigrants_2017.drop("District Code", axis=1)

# Grouping the number of nationalities by District Name
immigrants_district = immigrants_2017.groupby(["District Name", "Nationality"])["Number"].sum().reset_index()

# Removing the 0 values from the table
immigrants_district_null = immigrants_district.loc[(immigrants_district["Number"]!=0)]

# Dropping "No consta" records and "Spain" as nationality
df_nationality = immigrants_district_null[(immigrants_district_null["District Name"]!="No consta") & (immigrants_district_null["Nationality"]!="Spain")]

# Get Top 3
df_nationality3 = df_nationality.groupby('District Name', as_index=False).apply(lambda x: x.sort_values(by='Number',ascending=False).head(3)).reset_index(drop=True)

df_nationality3

,District Name,Nationality,Number
0,Ciutat Vella,Italy,3410
1,Ciutat Vella,Pakistan,3242
2,Ciutat Vella,France,1548
3,Eixample,Italy,4043
4,Eixample,China,2674
5,Eixample,Colombia,1844
6,Gràcia,Italy,1554
7,Gràcia,France,766
8,Gràcia,Venezuela,498
9,Horta-Guinardó,Italy,971


In [198]:
# Checking if total number of immigrants in both datasets match
df_tosum = immigrants_district_null[(immigrants_district_null["District Name"]!="No consta") & (immigrants_district_null["Nationality"]!="Spain")]
df_check = df_tosum.groupby("District Name", as_index=False).agg({"Number":"sum"})

df_p = df_population.reset_index()
df_check["#Immigrants"] = df_p["#Immigrants"]
df_check["#Inhabitants"] = df_p["#Immigrants"]
df_check["% between datasets"] = (df_check["#Immigrants"]*100/df_check["Number"]).round(2)
df_check
#average_diff = sum(df_check["% between datasets"])/10
#average_diff


# There is a difference of approximately 60 % between the total number of immigrants
# recorded in the dataset immigrants_by_nationality.csv and those in the
# 2017_immigrants_sexe.csv dataset. The source of the second (Open Data BCN) explains
# that:
#
# The data origins from the difference between the registers (because of birth,
# immigration or omission) and the quitting (because of decease, emigration,
# undue inclusion or duplicity) accounted for in the population register of the city
# of Barcelona for a year.
#
# It is possible that the exclusion of these numbers accounts for the difference in values.


,District Name,Number,#Immigrants,#Inhabitants,% between datasets
0,Ciutat Vella,27573,12611,12611,45.74
1,Eixample,33233,19047,19047,57.31
2,Gràcia,11511,7254,7254,63.02
3,Horta-Guinardó,11592,7799,7799,67.28
4,Les Corts,6450,4375,4375,67.83
5,Nou Barris,13524,8274,8274,61.18
6,Sant Andreu,9510,6335,6335,66.61
7,Sant Martí,21180,12720,12720,60.06
8,Sants-Montjuïc,20088,11683,11683,58.16
9,Sarrià-Sant Gervasi,10385,7227,7227,69.59


In [199]:
# GET PERCENTAGE IMMIGRANTS IN RELATION TO TOTAL IMMIGRANTS

In [200]:
df_nationality3.set_index("District Name", inplace=True)
df_check.set_index("District Name", inplace=True)
df_nationality3["Total immigrants"] = df_check["Number"]
df_nationality3["Percent of immigrants/district"] = (df_nationality3["Number"]*100/df_nationality3["Total immigrants"]).round(2)

df_nationality3 = df_nationality3.drop(["Total immigrants"], axis=1)

df_nationality3

,Nationality,Number,Percent of immigrants/district
District Name,,,
Ciutat Vella,Italy,3410,12.37
Ciutat Vella,Pakistan,3242,11.76
Ciutat Vella,France,1548,5.61
Eixample,Italy,4043,12.17
Eixample,China,2674,8.05
Eixample,Colombia,1844,5.55
Gràcia,Italy,1554,13.50
Gràcia,France,766,6.65
Gràcia,Venezuela,498,4.33


In [ ]:
# GET CITY OVERALL STATS

In [208]:
df_nationality_city = df_nationality.copy()
df_nationality_city = df_nationality_city.groupby(["Nationality"], as_index=False).agg({"Number": "sum"})

total_immigrants_city = sum(df_check["Number"])
df_nationality_city["Percentage"] = (df_nationality_city["Number"]*100/total_immigrants_city).round(2)
df_nationality_city = df_nationality_city.sort_values(by=["Percentage"], ascending=False)
df_nationality_city

,Nationality,Number,Percentage
74,Italy,16091,9.75
118,Pakistan,9707,5.88
31,China,9349,5.66
32,Colombia,7777,4.71
52,France,7109,4.31
65,Honduras,6734,4.08
171,Venezuela,6642,4.02
104,Morocco,5574,3.38
122,Peru,5496,3.33
20,Brasil,4679,2.83
